# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
# It comes with the tensorflow-datasets module, therefore, if you haven't please install the package using
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data

That's where we load and preprocess our data.

In [2]:
# remember the comment from above
# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

# tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
# in our case, we are interesteed in the MNIST; the name of the dataset is the only mandatory argument
# there are other arguments we can specify, which we can find useful
# mnist_dataset = tfds.load(name='mnist', as_supervised=True)
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 

2021-10-27 15:02:09.764831: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1* mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation, tf.int64)

num_test_samples =  mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

#scale images
def scale(image,label):
    image = tf.cast(image, tf.float64)
    image /= 255.
    return image,label

scale_train_validation_data = mnist_train.map(scale)
scale_test_data = mnist_test.map(scale)

# let's also shuffle the data
BUFFER_SIZE = 10000

shuffled_scale_train_validation_data= scale_train_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_scale_train_validation_data.take(num_validation_samples)

train_data = shuffled_scale_train_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)
test_data = scale_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


2021-10-27 19:57:58.396207: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-27 19:57:59.480928: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


#    Outline the mode

In [44]:
input_size = 784
output_size = 10
hide_layer_size = 200


model = tf.keras.Sequential([
                        tf.keras.layers.Flatten(input_shape = (28,28,1)),
                        tf.keras.layers.Dense(hide_layer_size, activation= 'relu'),
                        tf.keras.layers.Dense(hide_layer_size, activation= 'relu'),
                        tf.keras.layers.Dense(output_size,  activation= 'softmax')   
])



# Choose the optimizer and the loss function

In [45]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model

In [46]:
NUM_EPOCHS= 5
model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose= 2)

Epoch 1/5
540/540 - 5s - loss: 0.2742 - accuracy: 0.9214 - val_loss: 0.1505 - val_accuracy: 0.9558
Epoch 2/5
540/540 - 3s - loss: 0.1069 - accuracy: 0.9679 - val_loss: 0.0911 - val_accuracy: 0.9728
Epoch 3/5
540/540 - 4s - loss: 0.0704 - accuracy: 0.9786 - val_loss: 0.0725 - val_accuracy: 0.9767
Epoch 4/5
540/540 - 3s - loss: 0.0530 - accuracy: 0.9832 - val_loss: 0.0523 - val_accuracy: 0.9845
Epoch 5/5
540/540 - 3s - loss: 0.0413 - accuracy: 0.9873 - val_loss: 0.0481 - val_accuracy: 0.9838


In [47]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 358ms/step - loss: 0.0800 - accuracy: 0.9757


In [48]:
print('Test loss: {0:.2f}. Test acuuracy: {1:.2f}'.format(test_loss, test_accuracy *100))

Test loss: 0.08. Test acuuracy: 97.57
